In [12]:
import pandas as pd
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel


/home/teo/dev/narxmodel/narx_darts/venv/lib/python3.10/site-packages/statsforecast/core.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
df = pd.read_csv('datasets/ph_dataset.csv', header=None, names=['x1', 'x2','y'])
data = df[:20]
data

,x1,x2,y
0,8.875837,1.0,6.912909
1,11.025642,1.0,10.839350
2,9.461355,1.0,10.894930
3,11.930201,1.0,10.955416
4,10.890642,1.0,10.969890
5,11.013423,1.0,10.984813
6,10.606074,1.0,10.999915
7,10.526539,1.0,11.016242
8,11.538829,1.0,11.029860
9,9.090840,1.0,11.042747


In [6]:
# Create the target time series
target_series = TimeSeries.from_dataframe(df, value_cols='y')

# Create the covariate time series
covariate_series = TimeSeries.from_dataframe(df, value_cols=['x1', 'x2'])

In [11]:
import joblib
scaler_target = joblib.load('scaler_target.save')
scaler_covariate = joblib.load('scaler_covariate.save')

# Scale real data
target_scaled = scaler_target.fit_transform(target_series)
covariate_scaled = scaler_covariate.transform(covariate_series)


In [17]:
model = RNNModel.load('model_futurecov')
predictions = model.predict(
   n=5,  series=target_scaled[:10], future_covariates=covariate_scaled
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/teo/dev/narxmodel/narx_darts/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 76.39it/s]


In [19]:
predictions_real = scaler_target.inverse_transform(predictions)
predictions_real

<TimeSeries (DataArray) (time: 5, component: 1, sample: 1)> Size: 40B
array([[[11.14258475]],

       [[11.24126753]],

       [[11.25133821]],

       [[11.3103636 ]],

       [[11.31991208]]])
Coordinates:
  * time       (time) int64 40B 10 11 12 13 14
  * component  (component) object 8B 'y'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None